In [1]:
import pandas as pd
import numpy as np

In [2]:
df_resale = pd.read_csv('../data/cleaned/resale_price_cleaned.csv')

In [3]:
df_resale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201060 entries, 0 to 201059
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   month                       201060 non-null  object 
 1   town                        201060 non-null  object 
 2   flat_type                   201060 non-null  int64  
 3   storey_range                201060 non-null  int64  
 4   floor_area_sqm              201060 non-null  float64
 5   flat_model                  201060 non-null  object 
 6   remaining_lease             201060 non-null  float64
 7   resale_price                196986 non-null  float64
 8   lat                         201060 non-null  float64
 9   lon                         201060 non-null  float64
 10  nearest_mrt_distance        201060 non-null  float64
 11  nearest_bus_distance        201060 non-null  float64
 12  education_score             201060 non-null  float64
 13  shopping_score

### Add inflation-adjusted resale price
Real Price = Nominal Price * (CPI Base Year/ CPI Curret Month)

In [3]:

df_deflated_resale = df_resale.copy()
cpi_base = 100
df_deflated_resale["deflated_resale_price"] = df_deflated_resale["resale_price"] * (100/ df_deflated_resale["CPI (base 2024-12)"])

### Drop correlated variables

In [4]:
df_deflated_resale.drop(columns = ['resale_price', 'flat_type', 'building_age_2025', 'total_unemployment_rate', 'flat_model',
                                   'CPI (base 2024-12)'], inplace = True)

In [21]:
df_deflated_resale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201060 entries, 0 to 201059
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   month                       201060 non-null  object 
 1   town                        201060 non-null  object 
 2   storey_range                201060 non-null  int64  
 3   floor_area_sqm              201060 non-null  float64
 4   remaining_lease             201060 non-null  float64
 5   lat                         201060 non-null  float64
 6   lon                         201060 non-null  float64
 7   nearest_mrt_distance        201060 non-null  float64
 8   nearest_bus_distance        201060 non-null  float64
 9   education_score             201060 non-null  float64
 10  shopping_score              201060 non-null  float64
 11  food_score                  201060 non-null  float64
 12  recreation_score            201060 non-null  float64
 13  healthcare_sco

In [9]:
import os

current_directory = os.getcwd()
print(current_directory)

/Users/judy/Desktop/DSE3101-Project/DSE3101-HDB-Decode/code


### Add ethnicity and religion columns 

In [5]:
# from another dataset
df_2 = pd.read_csv("../data/cleaned/resale_price_normalized_demo.csv")

In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196986 entries, 0 to 196985
Data columns (total 38 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         196986 non-null  object 
 1   town                          196986 non-null  object 
 2   flat_type                     196986 non-null  float64
 3   storey_range                  196986 non-null  float64
 4   floor_area_sqm                196986 non-null  float64
 5   flat_model                    196986 non-null  object 
 6   remaining_lease               196986 non-null  float64
 7   resale_price                  196986 non-null  float64
 8   lat                           196986 non-null  float64
 9   lon                           196986 non-null  float64
 10  nearest_mrt_distance          196986 non-null  float64
 11  nearest_bus_distance          196986 non-null  float64
 12  education_score               196986 non-nul

In [ ]:
# drop NoReligion and Others
df_com = pd.merge(df_deflated_resale, df_2[['town','Buddhism','Taoism1','Islam','Hinduism','Sikhism',
                                            'Christianity_Catholic','Christianity_OtherChristians','OtherReligions',
                                            'Chinese', 'Malays','Indians']], on = 'town', how = 'left')

In [31]:
unique_towns = df_2['town'].unique()
print(unique_towns)

['BUKIT MERAH' 'CENTRAL AREA' 'KALLANG/WHAMPOA' 'QUEENSTOWN'
 'MARINE PARADE' 'CLEMENTI' 'GEYLANG' 'BUKIT TIMAH' 'JURONG EAST' 'BEDOK'
 'JURONG WEST' 'TOA PAYOH' 'TAMPINES' 'BUKIT BATOK' 'BISHAN' 'SERANGOON'
 'HOUGANG' 'ANG MO KIO' 'PASIR RIS' 'BUKIT PANJANG' 'CHOA CHU KANG'
 'SENGKANG' 'PUNGGOL' 'YISHUN' 'WOODLANDS' 'SEMBAWANG']


In [28]:
df_2.unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

### Standardization (z-score)

In [25]:
df_resale_standardized = df_deflated_resale.copy()
def standardized(col):
    return (col - col.mean()) / col.std()

columns_to_standardized = df_deflated_resale.select_dtypes(include=['float64', 'int64']).columns.difference(['lat', 'lon'])
df_resale_standardized[columns_to_standardized] = df_deflated_resale[columns_to_standardized].apply(standardized)
df_resale_standardized.head()


,month,town,storey_range,floor_area_sqm,remaining_lease,lat,lon,nearest_mrt_distance,nearest_bus_distance,education_score,shopping_score,food_score,recreation_score,healthcare_score,inflation_rate (x100),resident_unemployment_rate,interest_rate,fx_rate,avg_household_income,deflated_resale_price
0,2017-03,BUKIT MERAH,0.883145,-1.579736,-1.161298,1.27038,103.823236,-0.099206,-0.247657,-1.847508,-0.655409,1.238303,-0.871097,-0.081186,-0.359725,-0.247514,-0.847830,1.912070,1.141354,-1.070912
1,2017-05,BUKIT MERAH,-0.127485,-1.579736,-1.172650,1.27038,103.823236,-0.099206,-0.247657,-1.847508,-0.655409,1.238303,-0.871097,-0.081186,0.344244,-0.247514,-0.933808,1.470345,1.141354,-1.072263
2,2018-01,BUKIT MERAH,0.377830,-1.579736,-1.220187,1.27038,103.823236,-0.099206,-0.247657,-1.847508,-0.655409,1.238303,-0.871097,-0.081186,-0.908394,-0.529231,-0.658519,-1.418779,1.141354,-1.171669
3,2018-06,BUKIT MERAH,-1.138114,-1.579736,-1.243601,1.27038,103.823236,-0.099206,-0.247657,-1.847508,-0.655409,1.238303,-0.871097,-0.081186,-0.324608,-0.529231,-0.285851,-0.407984,1.141354,-1.357525
4,2018-06,BUKIT MERAH,-0.632799,-1.579736,-1.243601,1.27038,103.823236,-0.099206,-0.247657,-1.847508,-0.655409,1.238303,-0.871097,-0.081186,-0.324608,-0.529231,-0.285851,-0.407984,1.141354,-1.323876


### One-Hot encoding

In [26]:
# drop the first value to avoid dummy variable trap
df_encoded = pd.get_dummies(df_resale_standardized, columns=['town'], drop_first=True)

In [27]:
df_encoded.to_csv('../data/cleaned/standardized_encoded_delfated_resale_price.csv', index=False)